# Nifty Daily data

In [2]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import talib
from talib import abstract

In [3]:
def get_missing_data_dates(stock_name,stock_data):
    
    holidays_file = "../Data/misc/holidays.csv"
    traded_on_holidays_file = "../Data/misc/traded_on_holidays.csv"
    traded_on_weekends_file = "../Data/misc/traded_on_weekends.csv"
    missing_data_file = "../Data/misc/missing_data.csv"
    
    stock_df = stock_data.copy()
    stock_df.drop(columns=['stock_code'],inplace=True)
    ## convert date to datetime format and make it as index
    stock_df['datetime'] =  pd.to_datetime(stock_df['datetime'], infer_datetime_format=True)
    stock_df = stock_df.set_index("datetime")

    ## Convert to  daily continous data
    stock_df_continous = stock_df.groupby(pd.Grouper(freq='1D')).agg({"open": "first", 
                                             "high": "max", 
                                             "low": "min", 
                                             "close": "last",
                                             "volume":"sum"})
    ## remove null data dates from continous data and capture those rows to na_data
    na_data = stock_df_continous[~stock_df_continous['open'].notna()]
    stock_df_continous = stock_df_continous[stock_df_continous['open'].notna()]
    stock_df_continous = stock_df_continous.sort_index(ascending=True)

    ## Data present on weekends
    traded_on_weekends = stock_df_continous[stock_df_continous.index.day_name().isin(['Saturday', 'Sunday'])]
    print(traded_on_weekends.shape)

    ## Convert holidays data to datetime and make it as index
    holidays = pd.read_csv(holidays_file)
    holidays['date'] = holidays['date'].astype('datetime64[ns]')
    holidays = holidays.set_index("date")

    ## Traded on holidays
    traded_on_holidays = stock_df_continous[stock_df_continous.index.isin(holidays.index)]

    ## remove weekends from null data of continous data
    na_data = na_data[~na_data.index.day_name().isin(['Saturday', 'Sunday'])]
    missing_data = na_data[~na_data.index.isin(holidays.index)]

    if os.path.exists(traded_on_holidays_file):
        a = pd.read_csv(traded_on_holidays_file)
    else:
        a = pd.DataFrame()
    if stock_name not in  a.columns:
        a[stock_name]= pd.Series([str(t) for t in traded_on_holidays.index.date])

    if os.path.exists(traded_on_weekends_file):
        b = pd.read_csv(traded_on_weekends_file)
    else:
        b = pd.DataFrame()
    if stock_name not in  b.columns:
        b[stock_name]= pd.Series([str(t) for t in traded_on_weekends.index.date])

    if os.path.exists(missing_data_file):
        c = pd.read_csv(missing_data_file)
    else:
        c = pd.DataFrame()
    if stock_name not in  c.columns:
        c[stock_name]= pd.Series([str(t) for t in missing_data.index.date])

    #a.to_csv(traded_on_holidays_file,index=False)
    #b.to_csv(traded_on_weekends_file,index=False)
    #c.to_csv(missing_data_file,index=False)

    return stock_df_continous


In [4]:
nifty_daily = pd.read_csv("../Data/Nifty/nifty_1day.csv")
nifty_daily_continous = get_missing_data_dates("Nifty",nifty_daily)
reliance_daily = pd.read_csv("../Data/Reliance/reliance_1day.csv")
reliance_daily_continous = get_missing_data_dates("Reliance",reliance_daily)

(14, 5)
(14, 5)


In [118]:
nifty_daily = pd.read_csv("../Data/Nifty/nifty_1day.csv")
nifty_daily.drop(columns=['stock_code'],inplace=True)
nifty_daily['datetime'] =  pd.to_datetime(nifty_daily['datetime'], infer_datetime_format=True)
cols = ['datetime','open','high','low','close']
# data = data[cols]
nifty_daily = nifty_daily.set_index("datetime")

In [120]:
nifty_daily_continous = nifty_daily.groupby(pd.Grouper(freq='1D')).agg({"open": "first", 
                                             "high": "max", 
                                             "low": "min", 
                                             "close": "last",
                                             "volume":"sum"})
na_data = nifty_daily_continous[~nifty_daily_continous['open'].notna()]
nifty_daily_continous = nifty_daily_continous[nifty_daily_continous['open'].notna()]
nifty_daily_continous = nifty_daily_continous.sort_index(ascending=True)

In [121]:
na_data

,open,high,low,close,volume
datetime,,,,,
2007-01-06,NaN,NaN,NaN,NaN,0
2007-01-07,NaN,NaN,NaN,NaN,0
2007-01-11,NaN,NaN,NaN,NaN,0
2007-01-13,NaN,NaN,NaN,NaN,0
2007-01-14,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...
2022-10-26,NaN,NaN,NaN,NaN,0
2022-10-29,NaN,NaN,NaN,NaN,0
2022-10-30,NaN,NaN,NaN,NaN,0


In [57]:
nifty_daily_continous

,open,high,low,close,volume
datetime,,,,,
2007-01-02,3966.25,4016.65,3964.95,4007.40,277443911
2007-01-03,4007.70,4029.80,3985.00,4024.05,336972857
2007-01-04,4027.30,4041.00,3979.80,3988.80,369793444
2007-01-05,3990.15,4011.45,3969.65,3983.40,363396365
2007-01-08,3983.30,3987.45,3913.00,3933.40,345887697
...,...,...,...,...,...
2022-11-01,18130.70,18175.80,18060.15,18145.40,248784199
2022-11-02,18177.90,18178.75,18048.65,18082.85,2053916079
2022-11-03,17968.35,18106.30,17959.20,18052.70,250997227


In [63]:
traded_on_weekends = nifty_daily_continous[nifty_daily_continous.index.day_name().isin(['Saturday', 'Sunday'])]
traded_on_weekends

,open,high,low,close,volume
datetime,,,,,
2009-10-17,5159.35,5176.80,5124.25,5141.80,210638083
2010-02-06,4712.75,4768.15,4712.75,4757.25,132000049
2012-01-07,4755.60,4759.40,4743.05,4746.90,90456550
2012-03-03,5360.05,5369.60,5353.40,5359.40,113199762
2012-04-28,5209.60,5216.40,5196.35,5209.00,77942285
2012-09-08,5343.65,5366.30,5343.45,5358.70,75696522
2013-05-11,6088.20,6114.55,6084.15,6107.25,75688341
2013-11-03,6332.05,6342.95,6311.15,6317.35,182578015
2014-03-22,6497.80,6502.65,6481.35,6494.90,90860320


In [58]:
## Holidays or weekends
holidays = pd.read_csv("../Data/holidays.csv")
trade_on_holiday = pd.read_csv("../Data/trade_on_holidays.csv")
trade_on_weekend = pd.read_csv("../Data/trade_on_weekends.csv")

In [59]:
holidays['date'] = holidays['date'].astype('datetime64[ns]')
holidays = holidays.set_index("date")

In [91]:
traded_on_holidays = nifty_daily_continous[nifty_daily_continous.index.isin(holidays.index)]
traded_on_holidays

,open,high,low,close,volume
datetime,,,,,
2007-11-09,5703.45,5794.20,5614.90,5663.25,177709487
2008-10-28,2526.20,2695.95,2526.20,2684.60,125137979
2009-03-27,3079.40,3123.35,3055.90,3108.65,879729349
2009-08-19,4457.75,4477.55,4353.45,4394.10,870511127
2010-11-05,6321.85,6338.50,6303.10,6312.45,239069253
2011-10-26,5214.95,5219.25,5196.15,5201.80,137221682
2012-10-26,5683.55,5697.20,5641.75,5664.30,625233411
2012-11-13,5689.70,5698.25,5660.35,5666.95,254963004
2013-11-14,6037.00,6101.65,6036.65,6056.15,527842208


In [84]:
holidays


,day,description
date,,
2007-01-01,Monday,Bakri ID
2007-01-26,Friday,Republic Day
2007-01-30,Tuesday,Moharram
2007-02-16,Friday,Mahashivratri
2007-03-27,Tuesday,Ram Navami
...,...,...
2022-08-15,Monday,Independence Day
2022-08-31,Wednesday,Ganesh Chaturthi
2022-10-05,Wednesday,Dussehra


In [61]:
na_data

,open,high,low,close,volume
datetime,,,,,
2007-01-06,NaN,NaN,NaN,NaN,0
2007-01-07,NaN,NaN,NaN,NaN,0
2007-01-11,NaN,NaN,NaN,NaN,0
2007-01-13,NaN,NaN,NaN,NaN,0
2007-01-14,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...
2022-10-26,NaN,NaN,NaN,NaN,0
2022-10-29,NaN,NaN,NaN,NaN,0
2022-10-30,NaN,NaN,NaN,NaN,0


In [86]:
na_data = na_data[~na_data.index.day_name().isin(['Saturday', 'Sunday'])]
na_data

,open,high,low,close,volume
datetime,,,,,
2007-01-11,NaN,NaN,NaN,NaN,0
2007-01-26,NaN,NaN,NaN,NaN,0
2007-01-30,NaN,NaN,NaN,NaN,0
2007-02-16,NaN,NaN,NaN,NaN,0
2007-02-26,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...
2022-08-09,NaN,NaN,NaN,NaN,0
2022-08-15,NaN,NaN,NaN,NaN,0
2022-08-31,NaN,NaN,NaN,NaN,0


In [90]:
missing_data = na_data[~na_data.index.isin(holidays.index)]
missing_data

,open,high,low,close,volume
datetime,,,,,
2007-01-11,NaN,NaN,NaN,NaN,0
2007-02-26,NaN,NaN,NaN,NaN,0
2007-03-16,NaN,NaN,NaN,NaN,0
2007-03-23,NaN,NaN,NaN,NaN,0
2007-04-27,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...
2017-07-21,NaN,NaN,NaN,NaN,0
2017-08-18,NaN,NaN,NaN,NaN,0
2017-08-24,NaN,NaN,NaN,NaN,0


In [100]:
l = missing_data.index.strftime('%Y-%m-%d')


In [104]:
[str(t) for t in missing_data.index.date]

77

In [3]:
## Installing TA-Lib
!python -m pip install TA_Lib-0.4.24-cp310-cp310-win_amd64.whl


Processing c:\users\amrit\intrade_ai\notebooks\ta_lib-0.4.24-cp310-cp310-win_amd64.whl
TA-Lib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [5]:
## function to get indicator values in dataframe
def indicator(data):
    stock_data = data.copy()
    stock_data['rsi'] = abstract.RSI(data, timeperiod=14)
    stock_data['ema'] = abstract.EMA(data, timeperiod=14)
    stock_data['adx'] = abstract.ADX(data, timeperiod=14)
    return stock_data

In [6]:
indicator_data = indicator(nifty_daily_continous)

In [11]:
indicator_data

,open,high,low,close,volume,rsi,ema,adx
datetime,,,,,,,,
2007-01-02,3966.25,4016.65,3964.95,4007.40,277443911,NaN,NaN,NaN
2007-01-03,4007.70,4029.80,3985.00,4024.05,336972857,NaN,NaN,NaN
2007-01-04,4027.30,4041.00,3979.80,3988.80,369793444,NaN,NaN,NaN
2007-01-05,3990.15,4011.45,3969.65,3983.40,363396365,NaN,NaN,NaN
2007-01-08,3983.30,3987.45,3913.00,3933.40,345887697,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-11-01,18130.70,18175.80,18060.15,18145.40,248784199,69.030689,17658.226723,17.017707
2022-11-02,18177.90,18178.75,18048.65,18082.85,2053916079,66.453269,17714.843160,18.029645
2022-11-03,17968.35,18106.30,17959.20,18052.70,250997227,65.189792,17759.890739,18.368433


In [10]:
fig = px.line(x=indicator_data.index, y=indicator_data["adx"])
fig.show()